<a href="https://colab.research.google.com/github/apchavezr/18.-Modelado-y-simulaci-n/blob/main/simulacion_ventanilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulación discreta: sistema de atención en ventanilla

Este notebook simula un sistema con una única ventanilla donde los clientes llegan a intervalos aleatorios y son atendidos uno a uno. Los tiempos de llegada y de servicio siguen una distribución exponencial.

In [1]:
import heapq
import random

# Parámetros de la simulación
TIEMPO_MAXIMO = 5.0               # Tiempo total de simulación
TASA_LLEGADAS = 1.0               # Lambda = 1 llegada por unidad de tiempo
TASA_SERVICIO = 0.5               # Lambda = 0.5 servicio por unidad de tiempo

# Estado del sistema
reloj = 0.0
servidor_ocupado = False
cola = []

# Lista de eventos futuros (FEL): tuplas (tiempo, tipo_evento)
fel = []
heapq.heapify(fel)

# Métricas
eventos_log = []

In [2]:
def programar_evento(tiempo, tipo):
    heapq.heappush(fel, (tiempo, tipo))

def llegada():
    global servidor_ocupado, reloj

    eventos_log.append((reloj, 'Llegada', len(cola), 'Ocupado' if servidor_ocupado else 'Libre'))

    t_llegada = reloj + random.expovariate(TASA_LLEGADAS)
    if t_llegada <= TIEMPO_MAXIMO:
        programar_evento(t_llegada, 'llegada')

    if not servidor_ocupado:
        servidor_ocupado = True
        t_salida = reloj + random.expovariate(TASA_SERVICIO)
        programar_evento(t_salida, 'salida')
        eventos_log[-1] += (f"Salida en {round(t_salida,1)}",)
    else:
        cola.append(reloj)

def salida():
    global servidor_ocupado, reloj

    eventos_log.append((reloj, 'Salida', len(cola), 'Ocupado'))

    if cola:
        tiempo_llegada = cola.pop(0)
        t_salida = reloj + random.expovariate(TASA_SERVICIO)
        programar_evento(t_salida, 'salida')
        eventos_log[-1] += (f"Salida en {round(t_salida,1)}",)
    else:
        servidor_ocupado = False

In [3]:
# Programar primer evento de llegada
programar_evento(0.0, 'llegada')

# Bucle principal de simulación
while fel:
    reloj, tipo = heapq.heappop(fel)
    if reloj > TIEMPO_MAXIMO:
        break
    if tipo == 'llegada':
        llegada()
    elif tipo == 'salida':
        salida()

In [4]:
print(f"{'Tiempo':<8} {'Evento':<10} {'En cola':<8} {'Servidor':<10} {'Próximo evento':<15}")
for e in eventos_log:
    print(f"{e[0]:<8.1f} {e[1]:<10} {e[2]:<8} {e[3]:<10} {e[4] if len(e)>4 else ''}")

Tiempo   Evento     En cola  Servidor   Próximo evento 
0.0      Llegada    0        Libre      Salida en 0.1
0.1      Salida     0        Ocupado    
0.5      Llegada    0        Libre      Salida en 1.0
1.0      Salida     0        Ocupado    
4.5      Llegada    0        Libre      Salida en 6.0
